# FORK 6
## parallel_in_python3_fork6.ipynb

In [34]:
import sys
import os
from glob import glob
import numpy as np
from time import time
from cmlreaders import CMLReader
from cluster_helper.cluster import cluster_view

In [35]:
def run_pipeline1_parallel(input_dat):
    import sys
    sys.path.append('/home1/dscho/code/projects/connectivity_reinstatement')
    import connectivity_reinstatement
    import numpy as np
    
    try:
        subj, sess, lclz, mont = input_dat
        expmt = 'FR1'
        lclz = 0
        mont = 0
        freqs=np.logspace(np.log10(3), np.log10(110), num=22)
        connectivity_reinstatement.run_pipeline1(subj,
                                                 sess,
                                                 expmt=expmt,
                                                 lclz=lclz,
                                                 mont=mont,
                                                 freqs=freqs,
                                                 sleep_max=1200)
        return 1
    except:
        return 0

In [12]:
r1_df = CMLReader.get_data_index("r1")

In [15]:
test_subjs = ['R1223E', 'R1446T', 'R1416T', 'R1318N', 'R1020J', 'R1354E',
              'R1373T', 'R1336T', 'R1347D', 'R1377M', 'R1226D', 'R1146E',
              'R1003P', 'R1138T', 'R1286J', 'R1268T', 'R1032D', 'R1189M',
              'R1221P', 'R1134T', 'R1153T', 'R1089P', 'R1299T', 'R1170J',
              'R1342M', 'R1247P', 'R1274T', 'R1193T', 'R1277J', 'R1378T']
expmt = 'FR1'

In [17]:
input_dat = []
for subj in test_subjs:
    sessions = r1_df.query("(subject=='{}') & (experiment=='{}')".format(subj, expmt)).session.tolist()
    mont = r1_df.query("(subject=='{}') & (experiment=='{}')".format(subj, expmt)).montage.iat[0]
    lclz = r1_df.query("(subject=='{}') & (experiment=='{}')".format(subj, expmt)).localization.iat[0]

    n_mont = len(np.unique(r1_df.query("(subject=='{}') & (experiment=='{}')".format(subj, expmt)).montage))
    n_lclz = len(np.unique(r1_df.query("(subject=='{}') & (experiment=='{}')".format(subj, expmt)).localization))
    assert (n_mont==1) & (n_lclz==1)
    
    for sess in sessions:
        input_dat.append([subj, sess, lclz, mont])

In [32]:
input_dat_ = []
for row in input_dat:
    subj, sess, lclz, mont = row
    f = '/scratch/dscho/connectivity_reinstatement/data/events/encoding_retrieval_word_event_pairs-{}-ses{}.pkl'.format(subj, sess)
    if not os.path.exists(f):
        input_dat_.append(row)
print(len(input_dat), len(input_dat_))

65 49


In [36]:
# Parallel processing
print('Running code for {} sessions.\n'.format(len(input_dat_)))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=len(input_dat_), cores_per_job=4) as view: 
    output = view.map(run_pipeline1_parallel, input_dat_)

Running code for 49 sessions.

13 Engines running
Sending a shutdown signal to the controller and engines.


OSError: [Errno 16] Device or resource busy: '.nfs000000290091b82400000125'

In [3]:
ccg_dir = '/scratch/dscho/phase_precession'

sessions = ['U387_ses3', 'U369_env1', 'U372_ses2', 'U390_ses3', 
            'U369_env2', 'U373_ses1', 'U387_ses1', 'U380_ses2', 
            'U393_ses2', 'U395_ses2', 'U384_ses3', 'U385_ses4', 
            'U371_env2', 'U371_env3']
print('{} sessions to start\n'.format(len(sessions)))

completed_subjs = [os.path.basename(f)[0:9] for f in glob.glob('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/*morlet*.pkl')]
sessions = sessions[12:]
sessions = np.sort([subj for subj in sessions if subj not in completed_subjs])
print('{} uncompleted sessions to process:'.format(len(sessions)))
print(sessions)

14 sessions to start

2 uncompleted sessions to process:
['U371_env2' 'U371_env3']


In [ ]:
# Parallel processing
print('Running code for {} sessions.\n'.format(len(sessions)))
with cluster_view(scheduler="sge", queue="RAM.q", num_jobs=len(sessions), 
                  cores_per_job=4) as view: 
    output = view.map(calc_cross_electrode_phase_locking_mrl_morlet_parallel, sessions)

Running code for 2 sessions.

2 Engines running


In [7]:
# Serial processing
import sys
sys.path.append('/home1/dscho/code/projects/unit_activity_and_hpc_theta')
import phase_locking
output = phase_locking.calc_local_phase_locking(sessions[0])